# Average NL intensity
In the notebook, we'll try to compute the average NL intensity in Africa for 2015.

In [8]:
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image
from neal_utils import *
from jake_utils import check_batch_within_africa_jake
import glob
from osgeo import gdal, osr

In [9]:
#%matplotlib inline
%load_ext autoreload
%autoreload 2

#### Testing

In [ ]:
A = Image.open('/atlas/u/nj/viirs/2015/1/2/SVDNB_npp_20150101-20150131_75N060W_vcmcfg_v10_c201505111709.avg_rade9.tif')
A_count = Image.open('/atlas/u/nj/viirs/2015/1/2/SVDNB_npp_20150101-20150131_75N060W_vcmcfg_v10_c201505111709.cf_cvg.tif')
B = Image.open('/atlas/u/nj/viirs/2015/2/2/SVDNB_npp_20150201-20150228_75N060W_vcmcfg_v10_c201504281504.avg_rade9.tif')
B_count = Image.open('/atlas/u/nj/viirs/2015/2/2/SVDNB_npp_20150201-20150228_75N060W_vcmcfg_v10_c201504281504.cf_cvg.tif')

In [ ]:
print A.size
print A_count.size
print B.size
print B_count.size

In [ ]:
t0 = time.time()
A = np.array(A)
A_count = np.array(A_count)
B = np.array(B)
B_count = np.array(B_count)
t1 = time.time()
print 'Converted to arrays: {} seconds'.format(t1-t0)
print A.shape
print A_count.shape
print B.shape
print B_count.shape

In [ ]:
C = np.zeros((18000, 28800, 2))
C_count = np.zeros((18000, 28800, 2))

In [ ]:
t0 = time.time()
C[:,:,0] = A
C[:,:,1] = B
C_count[:,:,0] = A_count
C_count[:,:,1] = B_count
t1 = time.time()
print 'Filling 3D arrays: {} seconds'.format(t1-t0)

In [ ]:
# Using Rachel's code to combine them
t0 = time.time()
D, D_count = get_year_avg(C, C_count)
t1 = time.time()
print 'Averaged observations: {} seconds'.format(t1-t0)

In [ ]:
np.save('/atlas/u/nj/viirs/test.npy', D)
np.save('/atlas/u/nj/viirs/test_counts.npy', D_count)

### Averaging VIIRS data
Now that we know how to do what we want to do, let's write some code to go through the directories and average the observations for all 6 tiles over all 12 months. We'll average two months at a time since we know that works.

In [ ]:
data_dir = '/atlas/u/nj/viirs/2015/'

In [ ]:
t0 = time.time()
for tile in xrange(3,7):
    print 'Starting tile {}: {} seconds'.format(tile, time.time() - t0)
    for month in xrange(1, 13):
        print '  Starting month {}: {} seconds'.format(month, time.time() - t0)
        # If January, set running averages and counts to January data
        if month == 1:
            for fn in glob.glob(data_dir + str(month) + '/' + str(tile) + '/*'):
                if fn[-13:] == 'avg_rade9.tif':
                    avg_obs = Image.open(fn)
                    avg_obs = np.array(avg_obs)
                elif fn[-10:] == 'cf_cvg.tif':
                    total_counts = Image.open(fn)
                    total_counts = np.array(total_counts)
            print '    Loaded data for month {}: {} seconds'.format(month, time.time() - t0)
            rows, cols = avg_obs.shape
            print '      Rows: {}'.format(rows)
            print '      Cols: {}'.format(cols)
        # If any other month, average with previous observations and counts
        else:
            for fn in glob.glob(data_dir + str(month) + '/' + str(tile) + '/*'):
                if fn[-13:] == 'avg_rade9.tif':
                    month_obs = Image.open(fn)
                    month_obs = np.array(month_obs)
                elif fn[-10:] == 'cf_cvg.tif':
                    month_counts = Image.open(fn)
                    month_counts = np.array(month_counts)
            print '    Loaded data for month {}: {} seconds'.format(month, time.time() - t0)
            temp_obs = np.zeros((rows, cols, 2))
            temp_counts = np.zeros((rows, cols, 2))
            temp_obs[:,:,0] = avg_obs
            temp_counts[:,:,0] = total_counts
            temp_obs[:,:,1] = month_obs
            temp_counts[:,:,1] = month_counts
            avg_obs, total_counts = get_year_avg(temp_obs, temp_counts)
            print '    Averaged observations for month {}: {} seconds'.format(month, time.time() - t0)
    obs_fn = data_dir + 'tile' + str(tile) + '_obs.npy'
    counts_fn = data_dir + 'tile' + str(tile) + '_counts.npy'
    np.save(obs_fn, avg_obs)
    print '  Saved observations for tile {}: {} seconds'.format(tile, time.time() - t0)
    np.save(counts_fn, total_counts)
    print '  Saved counts for tile {}: {} seconds'.format(tile, time.time() - t0)

## Filter out points not in Africa
In order to decide how many classes of nightlights we want to have, we want to filter out all the points that aren't in Africa and plot a histogram.

#### Tile 2
Let's start with tile 2.

In [12]:
# Load geotif
geotif_addr = '../../cs231nproject/SVDNB_npp_20151201-20151231_75N060W_vcmcfg_v10_c201601251413.avg_rade9.tif'
geotif = Image.open(geotif_addr)
# Load averaged observations
#tile_obs_fn = '/atlas/u/nj/viirs/2015/data/tile2_obs.npy'
#tile_obs = np.load(tile_obs_fn)
rows, cols = geotif.shape
print 'Rows: {}'.format(rows)
print 'Columns: {}'.format(cols)

/Users/jakekim/anaconda/lib/python2.7/site-packages/PIL/Image.py:2221: DecompressionBombWarning: Image size (518400000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


AttributeError: shape

In [10]:
# Produce list of pixel pairs
pixel_pairs = make_pixel_pairs(tile_obs[:1000,:1000])

In [11]:
# Convert pixel pairs to lat/lon pairs
t0 = time.time()
lat_lon_pairs = pixel_to_lat_lon(geotif_addr, pixel_pairs)
print 'Took {} seconds'.format(time.time() - t0)

Took 2.1130900383 seconds


In [16]:
# Convert lat/lon pairs to arrays
lats = []
lons = []
t0 = time.time()
for lat, lon in lat_lon_pairs:
    lats.append(lat)
    lons.append(lon)
lats = np.array(lats)
lons = np.array(lons)
within = check_batch_within_africa(lats, lons)
t1 = time.time()
print 'Checked {} points in {} seconds'.format(within.size, t1-t0)

Checked 10000 points: 0.387810945511 seconds
Checked 20000 points: 0.769067049026 seconds
Checked 30000 points: 1.1513710022 seconds
Checked 40000 points: 1.53577899933 seconds
Checked 50000 points: 1.9137570858 seconds
Checked 60000 points: 2.29300713539 seconds
Checked 70000 points: 2.67170000076 seconds
Checked 80000 points: 3.04915595055 seconds
Checked 90000 points: 3.42779612541 seconds
Checked 100000 points: 3.80856895447 seconds
Checked 110000 points: 4.20006108284 seconds
Checked 120000 points: 4.59612298012 seconds
Checked 130000 points: 4.98421597481 seconds
Checked 140000 points: 5.36938214302 seconds
Checked 150000 points: 5.76044797897 seconds
Checked 160000 points: 6.14253902435 seconds
Checked 170000 points: 6.52360796928 seconds
Checked 180000 points: 6.90638995171 seconds
Checked 190000 points: 7.29086112976 seconds
Checked 200000 points: 7.68519711494 seconds
Checked 210000 points: 8.06682395935 seconds
Checked 220000 points: 8.44551992416 seconds
Checked 230000 poin

In [32]:
# Get nl values for pixels in Africa
pixel_pairs_in_africa = np.array(pixel_pairs)
nl_values = np.zeros((pixel_pairs_in_africa.shape[0],))
t0 = time.time()
for i in xrange(pixel_pairs_in_africa.shape[0]):
    nl_values[i] = tile_obs[pixel_pairs_in_africa[i,1], pixel_pairs_in_africa[i,0]]
    if i % 1000 == 0:
        print 'Finished {} points: {} seconds'.format(i, time.time() - t0)

Finished 0 points: 0.00038480758667 seconds
Finished 1000 points: 0.00233793258667 seconds
Finished 2000 points: 0.00344586372375 seconds
Finished 3000 points: 0.00438785552979 seconds
Finished 4000 points: 0.00534892082214 seconds
Finished 5000 points: 0.00627303123474 seconds
Finished 6000 points: 0.00719594955444 seconds
Finished 7000 points: 0.00828981399536 seconds
Finished 8000 points: 0.00920987129211 seconds
Finished 9000 points: 0.010617017746 seconds
Finished 10000 points: 0.0115609169006 seconds
Finished 11000 points: 0.0124850273132 seconds
Finished 12000 points: 0.0134248733521 seconds
Finished 13000 points: 0.0143489837646 seconds
Finished 14000 points: 0.0152719020844 seconds
Finished 15000 points: 0.0163638591766 seconds
Finished 16000 points: 0.0173029899597 seconds
Finished 17000 points: 0.0182268619537 seconds
Finished 18000 points: 0.019150018692 seconds
Finished 19000 points: 0.0200748443604 seconds
Finished 20000 points: 0.0209999084473 seconds
Finished 21000 poin

In [31]:
pixel_pairs_in_africa.shape

(1000000, 2)

In [33]:
nl_values

array([ 0.7136881 ,  0.70755659,  0.70322937, ...,  0.59321586,
        0.59358504,  0.6381932 ])